In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException,TimeoutException , ElementNotVisibleException
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from hdfs import InsecureClient


import pandas as pd
from datetime import datetime


In [14]:
print("a")

a


**list_country**

In [15]:
def get_country():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options = chrome_options)
    try:
        url = "https://www.flightradar24.com/data/airports"
        driver.get(url)
        # Find all links to countries
        country_links = driver.find_elements(By.CSS_SELECTOR, 'table#tbl-datatable a[href^="https://www.flightradar24.com/data/airports/"]')
        country = []
        # Extract and print the country names
        for country_link in country_links:
            country_name = country_link.get_attribute("title")
            country.append(country_name.replace(" ", "-").replace("(", "").replace(")", ""))
        country_set = set(country)
    finally:
        driver.quit()
        return (country_set)

In [16]:

def scrape_airport_data(country_url):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options = chrome_options)

    try:
        driver.get(country_url)

        # Find all airport links
        IATA = []
        airport_names = []
        #k = 0
        airport_links = driver.find_elements(By.CSS_SELECTOR, 'a[data-iata][data-lat][data-lon]')
        for airport_link in airport_links:
            airport_name = airport_link.text.strip().split('\n')[0]
            airport_iata = airport_link.get_attribute('data-iata')
            airport_icao = airport_name.split('(')[-1].split(')')[0]
            IATA.append(airport_iata)
            airport_names.append(airport_name)
        airport_names_cleaned = [airport.split(' (')[0] for airport in airport_names]
 
    finally:
        driver.quit()
    return IATA , airport_names_cleaned


In [17]:

def scrape_all_countries_airports(main_url):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options = chrome_options)

    try:
        driver.get(main_url)

        # Find all links to countries
        country_links = driver.find_elements(By.CSS_SELECTOR, 'table#tbl-datatable a[href^="https://www.flightradar24.com/data/airports/"]')
        #p = 0
        AEROPORT_IATA = {}
        for country_link in set(country_links):
            country_url = country_link.get_attribute("href")
            #print(country_url)
            for elem in scrape_airport_data(country_url)[0] : 
                AEROPORT_IATA[elem] = country_url[44:]

            #p = p+1
           # if p>10 :
               # break 
    finally:
        driver.quit()
    return AEROPORT_IATA


In [18]:
def click_button( driver , button_text):
            k = 0
            while (True and k <10):
                try:
                    k = k+1
                    # Wait for the button to be present and visible
                    button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f'//button[text()="{button_text}"]')))
                    # Click the button
                    button.click()
                    time.sleep(3)
                except ElementClickInterceptedException:
                        # If click is intercepted, wait for the iframe to disappear
                        WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, 'google_ads_iframe_/59302966/fr24_data_anchor_0')))
                except (ElementClickInterceptedException, ElementNotInteractableException, TimeoutException) as e:
                    break
                except (ElementNotVisibleException, TimeoutException):
                    # Break the loop if the button is not visible or timeout occurs
                    break 


In [19]:
def scrape_flight_data (airport_code) :
    options = Options()
    """options.add_argument("start-maximized")
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-application-cache')
    options.add_argument('--disable-gpu')
    options.add_argument("--disable-dev-shm-usage")"""
    options.add_argument("--headless")
    driver = webdriver.Chrome(options = options)
    airport_url = "https://www.flightradar24.com/data/airports/"+airport_code+"/departures"
    driver.maximize_window()
    driver.implicitly_wait(10)  # Attend jusqu'à 10 secondes avant de lancer une exception

    try:
        driver.get(airport_url)

        # Wait for the button to appear and accept cookies
        time.sleep(12)
        try:
            button1 = driver.find_element(By.ID, 'onetrust-accept-btn-handler')
            button1.click()
        except NoSuchElementException:
            pass  # If the accept cookies button is not found, continue without clicking
    
        """def click_button(ButtonName):
            k = 0
            while k < 10:
                try:
                    # Attente explicite pour l'élément
                    button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'btn-load-reviews')))
                    button.click()
                    k += 1
                except (ElementClickInterceptedException, ElementNotInteractableException):
                    break
                except TimeoutException:
                    # Si le bouton n'est pas trouvé dans le délai, sortir de la boucle
                    break"""            



        element = driver.find_elements(By.XPATH,'//*[@id="cnt-data-content"]/div/div[2]/div/aside/div[1]/table/tbody/tr[2]')
        for e in element:
            soup = e.get_attribute('outerHTML')
            sp = BeautifulSoup(soup, 'html.parser')
            tr_element = sp.find('tr')
            # Get the value of the class attribute
            class_value = tr_element.get('class')
            if (len(class_value) >0):
            # Wait for the "Load later flights" button to be clickable
                click_button(driver, "Load earlier flights") 
                #time.sleep(2)
                click_button(driver, "Load later flights") 
                #time.sleep(2)
                page_source = driver.page_source
                driver.quit()
                # Find all flight information rows
                soup2 = BeautifulSoup(page_source, 'html.parser')
                flight_rows = soup2.find_all('tr', class_='hidden-xs hidden-sm ng-scope')
                #flight_rows = soup2.find_elements(By.XPATH, '//tr[@class="hidden-xs hidden-sm ng-scope"]')
                flight_data = []
                for row in flight_rows:
                    data_date = row.get('data-date', 'N/A')
                    # Extract flight time
                    flight_time_tag = row.find('td', class_='ng-binding')
                    flight_time = flight_time_tag.text.strip() if flight_time_tag else 'N/A'

                    # Extract origin airport
                    origin_airport_tag = row.find('a', class_='fs-10 fbold notranslate ng-binding')
                   
                    arrival_airport = origin_airport_tag['title'] if origin_airport_tag else 'N/A'

                    # Extract flight number
                    flight_number_tag = row.find('a', class_='notranslate ng-binding')
                    flight_number = flight_number_tag.text.strip() if flight_number_tag else 'N/A'

                    # Extract aircraft model
                    aircraft_model_tag = row.find('span', class_='notranslate ng-binding')
                    aircraft_model = aircraft_model_tag.text.strip() if aircraft_model_tag else 'N/A'

                    # Extract flight status
                    flight_status_tag = row.find('span', class_='ng-binding', attrs={'ng-bind-html': 'objFlight.flight.statusMessage.text | unsafe'})
                    flight_status = flight_status_tag.text.strip() if flight_status_tag else 'N/A'

                    # Extract arrival time
                    arrival_time_td = row.find_all('td', class_='ng-binding')[1]
                    time_arrival = arrival_time_td.text.strip().split()[-1] if arrival_time_td else 'Time not found.'
                    # Extract aircraft
                    aircraft_tag_list = row.find_all('a', class_='notranslate ng-binding')
                    if len(aircraft_tag_list) >= 2:
                        aircraft = aircraft_tag_list[1].text.strip()
                    else:
                        aircraft = 'N/A'
                    # Append data to the list as a dictionary
                    flight_data.append({
        "Date": data_date,
        "Flight Time": flight_time,
        "Airline": aircraft,
        "Destination Aeroport": arrival_airport,
        "flight": flight_number,
        "Aircraft Model": aircraft_model,
        "Flight Status": flight_status,
        "time_arrival":time_arrival})
                df = pd.DataFrame(flight_data)
                return (df)
            else : 
                #print("we don't have any data for this ariport" , airport_code)
                return "we don't have any data for this ariport" 


    finally:
            driver.quit()
            

In [20]:
def scrape_flight_from_country (country_name):
    url_country = "https://www.flightradar24.com/data/airports/" + country_name
    IATA , aeroport_names = scrape_airport_data(url_country)
    data_total_country = []
    for i, j in  zip (IATA, aeroport_names) : 
        df_aeroport = scrape_flight_data(i) 
        if isinstance(df_aeroport, pd.DataFrame):
            print("data founded in ",j , " = " , len(df_aeroport)  )
            df_aeroport["Original Aeroport"] =  j 
            data_total_country.append(df_aeroport)
    if len(data_total_country) >0 :
        concatenated_df = pd.concat(data_total_country, ignore_index=True)
        if concatenated_df is not None : 
            return concatenated_df
    else :
        return "No data"


In [21]:
import os
import csv

def save_csv(data, parent_folder, folder_name, file_name):
    # Create the folder if it doesn't exist
    folder_path = os.path.join(parent_folder, folder_name)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Define the file path within the new folder
    file_path = os.path.join(folder_path, file_name)
    send_to_hdfs(data , file_path.replace(os.path.sep, '/'))


In [22]:
def send_to_hdfs(data, file_path): 
    client = InsecureClient('http://localhost:50070')
    with client.write(file_path, overwrite=True) as writer:
        data.to_csv(writer, index=False)


In [23]:
from IPython.display import HTML
def scrape_all_arrival_flight() :
    countries = get_country()
    #countries_done= ["France","Cocos-keeling-Islands","Martinique","Botswana","Belgium","Latvia","Guam","Chile","Senegal","Macao","Cameroon","Germany","Saint-Vincent-And-The-Grenadines","Eritrea", "Jersey","Luxembourg","Mali","Egypt","Estonia","Yemen", "Colombia","New-Zealand" , "Dominica","Guinea-bissau","Antigua-And-Barbuda","Puerto-Rico","Rwanda" ,"Faroe-Islands","Kuwait","Ivory-Coast","Djibouti","Guatemala","Romania","Canada","New-Zealand","Dominica"]
    for country in countries : 
     #   if country not in countries_done :
            print("start of extract data from :", country)
            data = scrape_flight_from_country(country) 
            print("End of extract data from :", country , " -----------------------------")
            #countries_done.append(country)
            # File path to save the CSV data

            date = datetime.now().date()
            # File path to save the CSV data
            file_name = country + str(date) +".csv"
            # Save the DataFrame to a CSV file
            parent_folder = "/user/PFE_data/departure_flights"
            #folder name 
            folder_name = country

            if isinstance(data, pd.DataFrame) :
                save_csv(data,parent_folder ,folder_name,file_name )
                message = '<p style="color:green;">data saved</p>'
                display(HTML(message))


In [24]:
scrape_all_arrival_flight()

start of extract data from : Benin
data founded in  Cotonou Cadjehoun Airport  =  30
End of extract data from : Benin  -----------------------------


start of extract data from : Belarus
data founded in  Brest Airport  =  1
data founded in  Gomel Airport  =  3
data founded in  Minsk National Airport  =  81
End of extract data from : Belarus  -----------------------------


start of extract data from : Burkina-Faso
End of extract data from : Burkina-Faso  -----------------------------
start of extract data from : Nauru
data founded in  Nauru International Airport  =  3
End of extract data from : Nauru  -----------------------------


start of extract data from : Singapore


In [ ]:
print("a")